In [3]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import sparse
import os
import matplotlib.pyplot as plt
import pickle
from scipy.sparse import csr_matrix

## Load data

In [2]:
rna_cite = ad.read_h5ad("../data_extra_modalities/rna_cite-pp-harm.h5ad")
adt_cite = ad.read_h5ad("../data_extra_modalities/adt_cite-pp-harm.h5ad")
cytof = ad.read_h5ad("../data_extra_modalities/cytof-pp-harm.h5ad")
rna_bulk = pd.read_csv("../data_extra_modalities/CBD-KEY-RNASEQ-WB/Raw_count_data_143_60683.txt", sep='\t')
luminex = pd.read_csv("../data_extra_modalities/CBD-KEY-LUMINEX/luminex.csv", sep=',')
clinvar = pd.read_csv("../data_extra_modalities/CBD-KEY-CLINVAR/COMBAT_CLINVAR_for_processed.txt", sep='\t')
gene_membership_cite = pd.read_csv("../data_extra_modalities/membership_all_celltypes.csv", sep='\t')
gene_membership_bulk = pd.read_csv("../data_extra_modalities/bulk_genes_membership.tsv", sep='\t')

In [6]:
rna_cite.obs

Index(['Annotation_cluster_id', 'Annotation_cluster_name',
       'Annotation_minor_subset', 'Annotation_major_subset',
       'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI',
       'QC_pct_mitochondrial', 'QC_scrub_doublet_scores',
       ...
       'Outcome', 'TimeSinceOnset', 'Ethnicity', 'Tissue',
       'DiseaseClassification', 'Pool_ID', 'Channel_ID', 'Domain',
       'Domain_major', 'Framework'],
      dtype='object', length=106)

In [3]:
#rna_cite.layers['raw'][]

In [4]:
rna_bulk

,S00016-Ja001T-TRGa,S00020-Ja003T-TRGa,S00024-Ja003T-TRGa,S00027-Ja003T-TRGa,S00028-Ja001T-TRGa,S00030-Ja003T-TRGa,S00033-Ja001T-TRGa,S00033-Ja003T-TRGa,S00034-Ja005T-TRGa,S00033-Ja005T-TRGa,...,S00081-Ja001T-TRGa,S00081-Ja005T-TRGa,S00082-Ja001T-TRGa,S00094-Ja005T-TRGa,S00095-Ja005T-TRGa,S00096-Ja005T-TRGa,S00097-Ja003T-TRGa,S00099-Ja005T-TRGa,S00104-Ja003T-TRGa,S00106-Ja003T-TRGa
ENSG00000000003,14,14,6,0,20,0,0,1,1,5,...,6,6,13,6,8,4,8,12,28,0
ENSG00000000005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000000419,708,629,734,488,1249,342,389,615,760,459,...,535,843,739,451,513,522,656,751,1017,438
ENSG00000000457,1203,787,943,820,1363,698,572,773,958,649,...,865,1356,1048,669,847,1242,868,942,1148,673
ENSG00000000460,293,430,280,135,502,212,139,186,215,144,...,260,286,351,196,277,215,229,344,283,174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288600,1,1,1,1,0,0,0,0,2,2,...,0,4,4,4,1,0,2,3,2,2
ENSG00000288601,0,0,0,0,0,0,0,0,0,1,...,3,1,0,0,0,0,1,2,0,0
ENSG00000288602,14,3,9,7,8,7,10,17,5,10,...,9,17,17,9,15,18,9,7,2,9
ENSG00000288603,6,4,3,0,0,5,4,6,7,3,...,3,5,2,4,5,5,1,1,4,6


## Prepare patient_id columns and save old ids

In [7]:
rna_cite.obs['patient_id'] = rna_cite.obs['scRNASeq_sample_ID'].copy()
adt_cite.obs['patient_id'] = adt_cite.obs['scRNASeq_sample_ID'].copy()
cytof.obs['patient_id_old'] = cytof.obs['patient_id'].copy()

NameError: name 'adt_cite' is not defined

In [8]:
rna_cite.obs[['patient_id','Source']]

,patient_id,Source
AAACCTGAGAAAGTGG-1-gPlexA1,S00109-Ja001E-PBCa,COVID_SEV
AAACCTGAGCGGATCA-1-gPlexA1,S00112-Ja003E-PBCa,COVID_MILD
AAACCTGAGGACATTA-1-gPlexA1,G05153-Ja005E-PBCa,COVID_HCW_MILD
AAACCTGAGGCGACAT-1-gPlexA1,S00005-Ja005E-PBCa,COVID_CRIT
AAACCTGAGGGAACGG-1-gPlexA1,S00061-Ja003E-PBCa,COVID_SEV
...,...,...
TTTGTCAGTGGCAAAC-1-gPlexK7,H00064-Ha001E-PBGa,HV
TTTGTCAGTTACCGAT-1-gPlexK7,U00501-Ua005E-PBUa,Flu
TTTGTCATCCTCTAGC-1-gPlexK7,G05112-Ja005E-PBCa,COVID_HCW_MILD
TTTGTCATCGAGGTAG-1-gPlexK7,S00045-Ja001E-PBCa,COVID_SEV


## Create anndata for bulk rna assay

In [6]:
raw_matrix = rna_bulk.to_numpy().transpose()
features = rna_bulk.index
patients = rna_bulk.columns
rna_bulk = ad.AnnData(X = raw_matrix)
rna_bulk.var_names = features
rna_bulk.obs['patient_id'] = patients
rna_bulk

/tmp/ipykernel_1827733/2623117053.py:4: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  rna_bulk = ad.AnnData(X = raw_matrix)


AnnData object with n_obs × n_vars = 143 × 60683
    obs: 'patient_id'

## Create anndata for luminex assay

In [7]:
patients = luminex['Row']
raw_matrix = luminex.iloc[:,6:].to_numpy()
features = luminex.columns[6:]
luminex = ad.AnnData(X = raw_matrix, dtype=np.float32)
luminex.var_names = features
luminex.obs['patient_id'] = patients.tolist()

## Extract and save patient ids

In [8]:
rna_cite.obs['patient_id'] = [id[:6] for id in rna_cite.obs['patient_id']]
adt_cite.obs['patient_id'] = [id[:6] for id in adt_cite.obs['patient_id']]
rna_bulk.obs['patient_id'] = [id[:6] for id in rna_bulk.obs['patient_id']]
cytof.obs['patient_id'] = [id[:6] for id in cytof.obs['patient_id']]
luminex.obs['patient_id'] = [id[:6] for id in luminex.obs['patient_id']]

## Identify common patients and subset anndatas

In [9]:
l1, l2, l3, l4, l5 = (rna_cite.obs['patient_id'], adt_cite.obs['patient_id'], rna_bulk.obs['patient_id'], cytof.obs['patient_id'],
luminex.obs['patient_id'])

In [10]:
common_patients = pd.Series(list(set(l1).intersection(set(l2).intersection(set(l3).intersection(set(l4).intersection(set(l5)))))))
common_patients

0     S00034
1     S00024
2     S00002
3     S00112
4     S00063
       ...  
80    S00042
81    S00148
82    S00066
83    S00072
84    S00114
Length: 85, dtype: object

In [11]:
rna_cite = rna_cite[rna_cite.obs['patient_id'].isin(common_patients),:]
adt_cite = adt_cite[adt_cite.obs['patient_id'].isin(common_patients),:]
cytof = cytof[cytof.obs['patient_id'].isin(common_patients),:]
luminex = luminex[luminex.obs['patient_id'].isin(common_patients),:]
rna_bulk = rna_bulk[rna_bulk.obs['patient_id'].isin(common_patients),:]

In [12]:
(rna_cite.obs_names == adt_cite.obs_names).all()

True

In [13]:
rna_cite.shape, adt_cite.shape, cytof.shape, luminex.shape, rna_bulk.shape

((650802, 20615), (650802, 192), (4976067, 48), (120, 51), (103, 60683))

## Fix gene membership (cite) dataset column names

In [14]:
gene_membership_cite[:3]

,orig_group,module,gene_id,gene_name,membership,p.value,moduleprefix,Unnamed: 7
0,1,NK,greenyellow,ENSG00000063177,RPL18,0.942322,2.893015e-61,NK|MEgreenyellow
1,2,NK,greenyellow,ENSG00000148303,RPL7A,0.942299,2.961341e-61,NK|MEgreenyellow
2,3,NK,greenyellow,ENSG00000134419,RPS15A,0.941011,1.131047e-60,NK|MEgreenyellow


In [15]:
gene_membership_cite.drop("orig_group", inplace=True, axis=1)

In [16]:
cols_to_keep = ['cell_type']
cols_to_keep = cols_to_keep + gene_membership_cite.columns[:-1].tolist()
cols_to_keep

['cell_type',
 'module',
 'gene_id',
 'gene_name',
 'membership',
 'p.value',
 'moduleprefix']

In [17]:
gene_membership_cite.set_axis(cols_to_keep, inplace=True, axis=1)
gene_membership_cite

/tmp/ipykernel_1827733/4182917993.py:1: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  gene_membership_cite.set_axis(cols_to_keep, inplace=True, axis=1)


,cell_type,module,gene_id,gene_name,membership,p.value,moduleprefix
0,NK,greenyellow,ENSG00000063177,RPL18,0.942322,2.893015e-61,NK|MEgreenyellow
1,NK,greenyellow,ENSG00000148303,RPL7A,0.942299,2.961341e-61,NK|MEgreenyellow
2,NK,greenyellow,ENSG00000134419,RPS15A,0.941011,1.131047e-60,NK|MEgreenyellow
3,NK,greenyellow,ENSG00000171863,RPS7,0.937765,2.908954e-59,NK|MEgreenyellow
4,NK,greenyellow,ENSG00000186468,RPS23,0.937428,4.032495e-59,NK|MEgreenyellow
...,...,...,...,...,...,...,...
78772,CD8,black,ENSG00000131759,RARA,0.164855,6.088533e-02,CD8|MEblack
78773,CD8,black,ENSG00000006194,ZNF263,0.156948,7.454333e-02,CD8|MEblack
78774,CD8,black,ENSG00000274588,DGKK,0.106502,2.278111e-01,CD8|MEblack
78775,CD8,black,ENSG00000154240,CEP112,0.103992,2.390201e-01,CD8|MEblack


## Explore  gene membership (cite) dataset

In [18]:
len(pd.unique(gene_membership_cite.gene_name))

13486

In [19]:
len(gene_membership_cite)

78777

In [20]:
sum(gene_membership_cite['gene_name'].isna())

34

In [21]:
np.where(gene_membership_cite['gene_name'] == 'RPL18')

(array([    0, 17470, 32529, 42924, 46451, 58392, 66735]),)

In [22]:
42731 / 650802

0.06565898691153377

In [23]:
len(np.where(rna_cite.obs.Annotation_major_subset == 'nan')[0])

42731

In [24]:
rna_cite.var_names

Index(['OR4F5', 'OR4F29', 'OR4F16', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1',
       'PERM1', 'HES4', 'ISG15',
       ...
       'AC007325.2', 'BX072566.1', 'AL354822.1', 'AC023491.2', 'AC004556.3',
       'AC233755.2', 'AC233755.1', 'AC240274.1', 'AC213203.4', 'AC213203.1'],
      dtype='object', length=20615)

In [25]:
len(rna_cite.var_names == 'nan')

20615

In [26]:
rna_cite[np.where(rna_cite.obs.Annotation_major_subset == 'nan')[0],]

View of AnnData object with n_obs × n_vars = 42731 × 20615
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_cont

In [27]:
np.where(len(gene_membership_cite.gene_name) == 0)

(array([], dtype=int64),)

In [28]:
gene_membership_cite.iloc[np.where(gene_membership_cite['gene_name'] == 'RPL18')]

,cell_type,module,gene_id,gene_name,membership,p.value,moduleprefix
0,NK,greenyellow,ENSG00000063177,RPL18,0.942322,2.893015e-61,NK|MEgreenyellow
17470,cMono,green,ENSG00000063177,RPL18,0.827735,3.585649e-34,cMono|MEgreen
32529,ncMono,brown,ENSG00000063177,RPL18,0.784924,1.350786e-28,ncMono|MEbrown
42924,PB,blue,ENSG00000063177,RPL18,0.887230,8.278263e-43,PB|MEblue
46451,B,magenta,ENSG00000063177,RPL18,0.905726,1.495188e-49,B|MEmagenta
58392,CD4,midnightblue,ENSG00000063177,RPL18,0.975066,7.223301e-87,CD4|MEmidnightblue
66735,CD8,magenta,ENSG00000063177,RPL18,0.816932,2.180619e-32,CD8|MEmagenta


In [29]:
gene_membership_cite.iloc[np.where(gene_membership_cite['cell_type'] == 'NK')]

,cell_type,module,gene_id,gene_name,membership,p.value,moduleprefix
0,NK,greenyellow,ENSG00000063177,RPL18,0.942322,2.893015e-61,NK|MEgreenyellow
1,NK,greenyellow,ENSG00000148303,RPL7A,0.942299,2.961341e-61,NK|MEgreenyellow
2,NK,greenyellow,ENSG00000134419,RPS15A,0.941011,1.131047e-60,NK|MEgreenyellow
3,NK,greenyellow,ENSG00000171863,RPS7,0.937765,2.908954e-59,NK|MEgreenyellow
4,NK,greenyellow,ENSG00000186468,RPS23,0.937428,4.032495e-59,NK|MEgreenyellow
...,...,...,...,...,...,...,...
12289,NK,salmon,ENSG00000118515,SGK1,0.265822,2.522888e-03,NK|MEsalmon
12290,NK,salmon,ENSG00000254174,IGHV1-12,0.264784,2.626763e-03,NK|MEsalmon
12291,NK,salmon,ENSG00000211654,IGLV5-37,0.238570,6.911488e-03,NK|MEsalmon
12292,NK,salmon,ENSG00000100290,BIK,0.200412,2.387278e-02,NK|MEsalmon


## Remove unused variables and cell_type-less observations

In [30]:
genes_not_in_assay = np.logical_not(gene_membership_cite.gene_name.isin(rna_cite.var_names))
genes_not_in_assay

0        False
1        False
2        False
3        False
4        False
         ...  
78772    False
78773    False
78774    False
78775    False
78776    False
Name: gene_name, Length: 78777, dtype: bool

In [31]:
sum(genes_not_in_assay)

324

In [32]:
gene_membership_cite.drop(np.where(genes_not_in_assay)[0], axis=0, inplace=True)

In [33]:
nan_cell_type_obs = rna_cite.obs.Annotation_major_subset == 'nan'
nan_cell_type_obs.any()

True

In [34]:
rna_cite = rna_cite[np.logical_not(nan_cell_type_obs),:].copy()
adt_cite = adt_cite[np.logical_not(nan_cell_type_obs),:].copy()

In [35]:
np.where(rna_cite.obs.Annotation_major_subset == 'nan')[0]

array([], dtype=int64)

In [36]:
nan_cell_type_obs = cytof.obs.Annotation_major_subset == 'nan'
nan_cell_type_obs.any()

True

In [37]:
cytof = cytof[np.logical_not(nan_cell_type_obs),:].copy()

In [38]:
np.where(rna_cite.obs.Annotation_major_subset == 'nan')[0]

array([], dtype=int64)

In [39]:
print(np.array([1,2,3,4]).sum(axis=0))
print(np.array([[1,2,3,4], [1,2,3,4]]).sum(axis=0))

10
[2 4 6 8]


## Generate rna_cite pseudobulks

In [40]:
rna_cite_pseudobulks = {}

for i, patient in enumerate(pd.unique(rna_cite.obs.patient_id)):
        rna_cite_patient = rna_cite[rna_cite.obs.patient_id == patient,:]
        for cell_type in pd.unique(rna_cite.obs.Annotation_major_subset):
            rna_cite_patient_cell_type = rna_cite_patient[rna_cite_patient.obs.Annotation_major_subset == cell_type, :]
            if len(rna_cite_patient_cell_type) > 0:
                for module in pd.unique(gene_membership_cite.module):
                    gene_membership_cite_module = gene_membership_cite.loc[gene_membership_cite.module == module, 'gene_name']
                    print(i, patient, cell_type, module)
                    #print(gene_membership_cite_module.shape)
                    #print(gene_membership_cite_module)
                    rna_cite_patient_cell_type_module = rna_cite_patient_cell_type[:,gene_membership_cite_module.tolist()]
                    #print(rna_cite_patient_cell_type_module.layers['raw'].data)
                    if rna_cite_patient_cell_type_module.shape[0] == 0:
                        raise ValueError("no observations for the current sum")
                    #print(rna_cite_patient_cell_type_module.shape)
                    #print(rna_cite_patient_cell_type_module)
                    #print(rna_cite_patient_cell_type_module.layers['raw'].shape)
                    #print(rna_cite_patient_cell_type_module.layers['raw'].sum(axis=0))
                    if np.isnan(rna_cite_patient_cell_type_module.layers['raw'].data).any():
                        raise ValueError("row contains nan care!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    
                    row = rna_cite_patient_cell_type_module.layers['raw'].toarray()
                    if row.shape[0] > 1:
                        row = row.sum(axis=0)
                    else:
                        row = row[0]
                    
                    #print(rna_cite_patient_cell_type.obs['patient_id'])
                    #if cell_type == 'iNKT' and module == 'greenyellow': 
                    #    print(row)
                    #print(type(row))
                    #print(row)
                    #print(row)
                    #print(np.isnan(row))
                    if np.any(np.isnan(row)):
                        #print(row)
                        raise ValueError("row contains nan care!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    key = f'{cell_type}-{module}'
                    print(row.shape)
                    if key not in rna_cite_pseudobulks:
                        df = pd.DataFrame(columns=gene_membership_cite_module)
                        df.loc[patient] = row
                        rna_cite_pseudobulks[key] = df
                    else:
                        rna_cite_pseudobulks[key].loc[patient] = row
                    
                #print(f'lenght of features in the cell type specific module: {len(gene_membership_cite_module)}')"""

0 S00109 NK greenyellow
(805,)
0 S00109 NK tan
(884,)
0 S00109 NK red
(897,)
0 S00109 NK turquoise
(20400,)
0 S00109 NK green
(10507,)
0 S00109 NK purple
(5277,)
0 S00109 NK brown
(4461,)
0 S00109 NK blue
(8022,)
0 S00109 NK black
(7643,)
0 S00109 NK salmon
(193,)
0 S00109 NK lightcyan
(778,)
0 S00109 NK grey60
(1750,)
0 S00109 NK grey
(3024,)
0 S00109 NK lightgreen
(263,)
0 S00109 NK pink
(5202,)
0 S00109 NK yellow
(2812,)
0 S00109 NK magenta
(2052,)
0 S00109 NK cyan
(2136,)
0 S00109 NK midnightblue
(1215,)
0 S00109 NK lightyellow
(132,)
0 S00109 CD8 greenyellow
(805,)
0 S00109 CD8 tan
(884,)
0 S00109 CD8 red
(897,)
0 S00109 CD8 turquoise
(20400,)
0 S00109 CD8 green
(10507,)
0 S00109 CD8 purple
(5277,)
0 S00109 CD8 brown
(4461,)
0 S00109 CD8 blue
(8022,)
0 S00109 CD8 black
(7643,)
0 S00109 CD8 salmon
(193,)
0 S00109 CD8 lightcyan
(778,)
0 S00109 CD8 grey60
(1750,)
0 S00109 CD8 grey
(3024,)
0 S00109 CD8 lightgreen
(263,)
0 S00109 CD8 pink
(5202,)
0 S00109 CD8 yellow
(2812,)
0 S00109 CD

(2812,)
0 S00109 PLT magenta
(2052,)
0 S00109 PLT cyan
(2136,)
0 S00109 PLT midnightblue
(1215,)
0 S00109 PLT lightyellow
(132,)
0 S00109 RET greenyellow
(805,)
0 S00109 RET tan
(884,)
0 S00109 RET red
(897,)
0 S00109 RET turquoise
(20400,)
0 S00109 RET green
(10507,)
0 S00109 RET purple
(5277,)
0 S00109 RET brown
(4461,)
0 S00109 RET blue
(8022,)
0 S00109 RET black
(7643,)
0 S00109 RET salmon
(193,)
0 S00109 RET lightcyan
(778,)
0 S00109 RET grey60
(1750,)
0 S00109 RET grey
(3024,)
0 S00109 RET lightgreen
(263,)
0 S00109 RET pink
(5202,)
0 S00109 RET yellow
(2812,)
0 S00109 RET magenta
(2052,)
0 S00109 RET cyan
(2136,)
0 S00109 RET midnightblue
(1215,)
0 S00109 RET lightyellow
(132,)
1 S00112 NK greenyellow
(805,)
1 S00112 NK tan
(884,)
1 S00112 NK red
(897,)
1 S00112 NK turquoise
(20400,)
1 S00112 NK green
(10507,)
1 S00112 NK purple
(5277,)
1 S00112 NK brown
(4461,)
1 S00112 NK blue
(8022,)
1 S00112 NK black
(7643,)
1 S00112 NK salmon
(193,)
1 S00112 NK lightcyan
(778,)
1 S00112 NK 

(5277,)
1 S00112 HSC brown
(4461,)
1 S00112 HSC blue
(8022,)
1 S00112 HSC black
(7643,)
1 S00112 HSC salmon
(193,)
1 S00112 HSC lightcyan
(778,)
1 S00112 HSC grey60
(1750,)
1 S00112 HSC grey
(3024,)
1 S00112 HSC lightgreen
(263,)
1 S00112 HSC pink
(5202,)
1 S00112 HSC yellow
(2812,)
1 S00112 HSC magenta
(2052,)
1 S00112 HSC cyan
(2136,)
1 S00112 HSC midnightblue
(1215,)
1 S00112 HSC lightyellow
(132,)
1 S00112 PLT greenyellow
(805,)
1 S00112 PLT tan
(884,)
1 S00112 PLT red
(897,)
1 S00112 PLT turquoise
(20400,)
1 S00112 PLT green
(10507,)
1 S00112 PLT purple
(5277,)
1 S00112 PLT brown
(4461,)
1 S00112 PLT blue
(8022,)
1 S00112 PLT black
(7643,)
1 S00112 PLT salmon
(193,)
1 S00112 PLT lightcyan
(778,)
1 S00112 PLT grey60
(1750,)
1 S00112 PLT grey
(3024,)
1 S00112 PLT lightgreen
(263,)
1 S00112 PLT pink
(5202,)
1 S00112 PLT yellow
(2812,)
1 S00112 PLT magenta
(2052,)
1 S00112 PLT cyan
(2136,)
1 S00112 PLT midnightblue
(1215,)
1 S00112 PLT lightyellow
(132,)
2 S00005 NK greenyellow
(805,)

2 S00005 GDT magenta
(2052,)
2 S00005 GDT cyan
(2136,)
2 S00005 GDT midnightblue
(1215,)
2 S00005 GDT lightyellow
(132,)
2 S00005 HSC greenyellow
(805,)
2 S00005 HSC tan
(884,)
2 S00005 HSC red
(897,)
2 S00005 HSC turquoise
(20400,)
2 S00005 HSC green
(10507,)
2 S00005 HSC purple
(5277,)
2 S00005 HSC brown
(4461,)
2 S00005 HSC blue
(8022,)
2 S00005 HSC black
(7643,)
2 S00005 HSC salmon
(193,)
2 S00005 HSC lightcyan
(778,)
2 S00005 HSC grey60
(1750,)
2 S00005 HSC grey
(3024,)
2 S00005 HSC lightgreen
(263,)
2 S00005 HSC pink
(5202,)
2 S00005 HSC yellow
(2812,)
2 S00005 HSC magenta
(2052,)
2 S00005 HSC cyan
(2136,)
2 S00005 HSC midnightblue
(1215,)
2 S00005 HSC lightyellow
(132,)
2 S00005 PLT greenyellow
(805,)
2 S00005 PLT tan
(884,)
2 S00005 PLT red
(897,)
2 S00005 PLT turquoise
(20400,)
2 S00005 PLT green
(10507,)
2 S00005 PLT purple
(5277,)
2 S00005 PLT brown
(4461,)
2 S00005 PLT blue
(8022,)
2 S00005 PLT black
(7643,)
2 S00005 PLT salmon
(193,)
2 S00005 PLT lightcyan
(778,)
2 S00005 

(778,)
3 S00061 DC grey60
(1750,)
3 S00061 DC grey
(3024,)
3 S00061 DC lightgreen
(263,)
3 S00061 DC pink
(5202,)
3 S00061 DC yellow
(2812,)
3 S00061 DC magenta
(2052,)
3 S00061 DC cyan
(2136,)
3 S00061 DC midnightblue
(1215,)
3 S00061 DC lightyellow
(132,)
3 S00061 GDT greenyellow
(805,)
3 S00061 GDT tan
(884,)
3 S00061 GDT red
(897,)
3 S00061 GDT turquoise
(20400,)
3 S00061 GDT green
(10507,)
3 S00061 GDT purple
(5277,)
3 S00061 GDT brown
(4461,)
3 S00061 GDT blue
(8022,)
3 S00061 GDT black
(7643,)
3 S00061 GDT salmon
(193,)
3 S00061 GDT lightcyan
(778,)
3 S00061 GDT grey60
(1750,)
3 S00061 GDT grey
(3024,)
3 S00061 GDT lightgreen
(263,)
3 S00061 GDT pink
(5202,)
3 S00061 GDT yellow
(2812,)
3 S00061 GDT magenta
(2052,)
3 S00061 GDT cyan
(2136,)
3 S00061 GDT midnightblue
(1215,)
3 S00061 GDT lightyellow
(132,)
3 S00061 HSC greenyellow
(805,)
3 S00061 HSC tan
(884,)
3 S00061 HSC red
(897,)
3 S00061 HSC turquoise
(20400,)
3 S00061 HSC green
(10507,)
3 S00061 HSC purple
(5277,)
3 S00061 

...

(2052,)
80 N00037 Mast cyan
(2136,)
80 N00037 Mast midnightblue
(1215,)
80 N00037 Mast lightyellow
(132,)
81 S00002 NK greenyellow
(805,)
81 S00002 NK tan
(884,)
81 S00002 NK red
(897,)
81 S00002 NK turquoise
(20400,)
81 S00002 NK green
(10507,)
81 S00002 NK purple
(5277,)
81 S00002 NK brown
(4461,)
81 S00002 NK blue
(8022,)
81 S00002 NK black
(7643,)
81 S00002 NK salmon
(193,)
81 S00002 NK lightcyan
(778,)
81 S00002 NK grey60
(1750,)
81 S00002 NK grey
(3024,)
81 S00002 NK lightgreen
(263,)
81 S00002 NK pink
(5202,)
81 S00002 NK yellow
(2812,)
81 S00002 NK magenta
(2052,)
81 S00002 NK cyan
(2136,)
81 S00002 NK midnightblue
(1215,)
81 S00002 NK lightyellow
(132,)
81 S00002 CD8 greenyellow
(805,)
81 S00002 CD8 tan
(884,)
81 S00002 CD8 red
(897,)
81 S00002 CD8 turquoise
(20400,)
81 S00002 CD8 green
(10507,)
81 S00002 CD8 purple
(5277,)
81 S00002 CD8 brown
(4461,)
81 S00002 CD8 blue
(8022,)
81 S00002 CD8 black
(7643,)
81 S00002 CD8 salmon
(193,)
81 S00002 CD8 lightcyan
(778,)
81 S00002 CD8

(10507,)
82 S00076 CD8 purple
(5277,)
82 S00076 CD8 brown
(4461,)
82 S00076 CD8 blue
(8022,)
82 S00076 CD8 black
(7643,)
82 S00076 CD8 salmon
(193,)
82 S00076 CD8 lightcyan
(778,)
82 S00076 CD8 grey60
(1750,)
82 S00076 CD8 grey
(3024,)
82 S00076 CD8 lightgreen
(263,)
82 S00076 CD8 pink
(5202,)
82 S00076 CD8 yellow
(2812,)
82 S00076 CD8 magenta
(2052,)
82 S00076 CD8 cyan
(2136,)
82 S00076 CD8 midnightblue
(1215,)
82 S00076 CD8 lightyellow
(132,)
82 S00076 ncMono greenyellow
(805,)
82 S00076 ncMono tan
(884,)
82 S00076 ncMono red
(897,)
82 S00076 ncMono turquoise
(20400,)
82 S00076 ncMono green
(10507,)
82 S00076 ncMono purple
(5277,)
82 S00076 ncMono brown
(4461,)
82 S00076 ncMono blue
(8022,)
82 S00076 ncMono black
(7643,)
82 S00076 ncMono salmon
(193,)
82 S00076 ncMono lightcyan
(778,)
82 S00076 ncMono grey60
(1750,)
82 S00076 ncMono grey
(3024,)
82 S00076 ncMono lightgreen
(263,)
82 S00076 ncMono pink
(5202,)
82 S00076 ncMono yellow
(2812,)
82 S00076 ncMono magenta
(2052,)
82 S00076 

(8022,)
82 S00076 Mast black
(7643,)
82 S00076 Mast salmon
(193,)
82 S00076 Mast lightcyan
(778,)
82 S00076 Mast grey60
(1750,)
82 S00076 Mast grey
(3024,)
82 S00076 Mast lightgreen
(263,)
82 S00076 Mast pink
(5202,)
82 S00076 Mast yellow
(2812,)
82 S00076 Mast magenta
(2052,)
82 S00076 Mast cyan
(2136,)
82 S00076 Mast midnightblue
(1215,)
82 S00076 Mast lightyellow
(132,)
83 S00072 NK greenyellow
(805,)
83 S00072 NK tan
(884,)
83 S00072 NK red
(897,)
83 S00072 NK turquoise
(20400,)
83 S00072 NK green
(10507,)
83 S00072 NK purple
(5277,)
83 S00072 NK brown
(4461,)
83 S00072 NK blue
(8022,)
83 S00072 NK black
(7643,)
83 S00072 NK salmon
(193,)
83 S00072 NK lightcyan
(778,)
83 S00072 NK grey60
(1750,)
83 S00072 NK grey
(3024,)
83 S00072 NK lightgreen
(263,)
83 S00072 NK pink
(5202,)
83 S00072 NK yellow
(2812,)
83 S00072 NK magenta
(2052,)
83 S00072 NK cyan
(2136,)
83 S00072 NK midnightblue
(1215,)
83 S00072 NK lightyellow
(132,)
83 S00072 CD8 greenyellow
(805,)
83 S00072 CD8 tan
(884,)
8

83 S00072 HSC yellow
(2812,)
83 S00072 HSC magenta
(2052,)
83 S00072 HSC cyan
(2136,)
83 S00072 HSC midnightblue
(1215,)
83 S00072 HSC lightyellow
(132,)
83 S00072 PLT greenyellow
(805,)
83 S00072 PLT tan
(884,)
83 S00072 PLT red
(897,)
83 S00072 PLT turquoise
(20400,)
83 S00072 PLT green
(10507,)
83 S00072 PLT purple
(5277,)
83 S00072 PLT brown
(4461,)
83 S00072 PLT blue
(8022,)
83 S00072 PLT black
(7643,)
83 S00072 PLT salmon
(193,)
83 S00072 PLT lightcyan
(778,)
83 S00072 PLT grey60
(1750,)
83 S00072 PLT grey
(3024,)
83 S00072 PLT lightgreen
(263,)
83 S00072 PLT pink
(5202,)
83 S00072 PLT yellow
(2812,)
83 S00072 PLT magenta
(2052,)
83 S00072 PLT cyan
(2136,)
83 S00072 PLT midnightblue
(1215,)
83 S00072 PLT lightyellow
(132,)
84 N00038 NK greenyellow
(805,)
84 N00038 NK tan
(884,)
84 N00038 NK red
(897,)
84 N00038 NK turquoise
(20400,)
84 N00038 NK green
(10507,)
84 N00038 NK purple
(5277,)
84 N00038 NK brown
(4461,)
84 N00038 NK blue
(8022,)
84 N00038 NK black
(7643,)
84 N00038 NK 

84 N00038 PLT tan
(884,)
84 N00038 PLT red
(897,)
84 N00038 PLT turquoise
(20400,)
84 N00038 PLT green
(10507,)
84 N00038 PLT purple
(5277,)
84 N00038 PLT brown
(4461,)
84 N00038 PLT blue
(8022,)
84 N00038 PLT black
(7643,)
84 N00038 PLT salmon
(193,)
84 N00038 PLT lightcyan
(778,)
84 N00038 PLT grey60
(1750,)
84 N00038 PLT grey
(3024,)
84 N00038 PLT lightgreen
(263,)
84 N00038 PLT pink
(5202,)
84 N00038 PLT yellow
(2812,)
84 N00038 PLT magenta
(2052,)
84 N00038 PLT cyan
(2136,)
84 N00038 PLT midnightblue
(1215,)
84 N00038 PLT lightyellow
(132,)


## Write rna_cite pseudobulk dictionary

In [41]:
with open('rna_cite_pseudobulks.pickle', 'wb') as f:
    pickle.dump(rna_cite_pseudobulks, f)

## Load rna_cite pseudobulk dictionary

In [42]:
with open("rna_cite_pseudobulks.pickle", "rb") as f:
    rna_cite_pseudobulks = pickle.load(f)

In [43]:
rna_cite_pseudobulks.keys()

dict_keys(['NK-greenyellow', 'NK-tan', 'NK-red', 'NK-turquoise', 'NK-green', 'NK-purple', 'NK-brown', 'NK-blue', 'NK-black', 'NK-salmon', 'NK-lightcyan', 'NK-grey60', 'NK-grey', 'NK-lightgreen', 'NK-pink', 'NK-yellow', 'NK-magenta', 'NK-cyan', 'NK-midnightblue', 'NK-lightyellow', 'CD8-greenyellow', 'CD8-tan', 'CD8-red', 'CD8-turquoise', 'CD8-green', 'CD8-purple', 'CD8-brown', 'CD8-blue', 'CD8-black', 'CD8-salmon', 'CD8-lightcyan', 'CD8-grey60', 'CD8-grey', 'CD8-lightgreen', 'CD8-pink', 'CD8-yellow', 'CD8-magenta', 'CD8-cyan', 'CD8-midnightblue', 'CD8-lightyellow', 'ncMono-greenyellow', 'ncMono-tan', 'ncMono-red', 'ncMono-turquoise', 'ncMono-green', 'ncMono-purple', 'ncMono-brown', 'ncMono-blue', 'ncMono-black', 'ncMono-salmon', 'ncMono-lightcyan', 'ncMono-grey60', 'ncMono-grey', 'ncMono-lightgreen', 'ncMono-pink', 'ncMono-yellow', 'ncMono-magenta', 'ncMono-cyan', 'ncMono-midnightblue', 'ncMono-lightyellow', 'cMono-greenyellow', 'cMono-tan', 'cMono-red', 'cMono-turquoise', 'cMono-green'

## Create adt_cite pseudobulks

In [44]:
(adt_cite.obs_names == rna_cite.obs_names).all()

True

In [87]:
adt_cite_pseudobulks = {}
for i, patient in enumerate(pd.unique(adt_cite.obs.patient_id)):
    adt_cite_patient = adt_cite[adt_cite.obs.patient_id == patient, :]
    for cell_type in pd.unique(adt_cite.obs.Annotation_major_subset):
        adt_cite_patient_cell_type = adt_cite_patient[adt_cite_patient.obs.Annotation_major_subset == cell_type, :]
        if len(adt_cite_patient_cell_type) > 0:
            print(i, patient, cell_type)
            key = f'{cell_type}'
            row = adt_cite_patient_cell_type.layers['raw'].toarray()
            if row.shape[0] > 1:
                row = row.sum(axis=0)
            else:
                row = row[0]
            if key not in adt_cite_pseudobulks:
                df = pd.DataFrame(columns = adt_cite.var_names)
                df.loc[patient] = row
                adt_cite_pseudobulks[key] = df
            else:
                adt_cite_pseudobulks[key].loc[patient] = row

0 S00109 NK
0 S00109 CD8
0 S00109 ncMono
0 S00109 cMono
0 S00109 CD4
0 S00109 B
0 S00109 MAIT
0 S00109 PB
0 S00109 iNKT
0 S00109 DN
0 S00109 DP
0 S00109 DC
0 S00109 GDT
0 S00109 HSC
0 S00109 PLT
0 S00109 RET
1 S00112 NK
1 S00112 CD8
1 S00112 ncMono
1 S00112 cMono
1 S00112 CD4
1 S00112 B
1 S00112 MAIT
1 S00112 PB
1 S00112 iNKT
1 S00112 DN
1 S00112 DP
1 S00112 DC
1 S00112 GDT
1 S00112 HSC
1 S00112 PLT
2 S00005 NK
2 S00005 CD8
2 S00005 ncMono
2 S00005 cMono
2 S00005 CD4
2 S00005 B
2 S00005 MAIT
2 S00005 PB
2 S00005 iNKT
2 S00005 DN
2 S00005 DP
2 S00005 DC
2 S00005 GDT
2 S00005 HSC
2 S00005 PLT
2 S00005 RET
3 S00061 NK
3 S00061 CD8
3 S00061 ncMono
3 S00061 cMono
3 S00061 CD4
3 S00061 B
3 S00061 MAIT
3 S00061 PB
3 S00061 iNKT
3 S00061 DN
3 S00061 DP
3 S00061 DC
3 S00061 GDT
3 S00061 HSC
3 S00061 PLT
4 S00056 NK
4 S00056 CD8
4 S00056 ncMono
4 S00056 cMono
4 S00056 CD4
4 S00056 B
4 S00056 MAIT
4 S00056 PB
4 S00056 iNKT
4 S00056 DN
4 S00056 DP
4 S00056 DC
4 S00056 GDT
4 S00056 HSC
4 S00056 PLT

40 S00003 NK
40 S00003 CD8
40 S00003 ncMono
40 S00003 cMono
40 S00003 CD4
40 S00003 B
40 S00003 MAIT
40 S00003 PB
40 S00003 iNKT
40 S00003 DN
40 S00003 DP
40 S00003 DC
40 S00003 GDT
40 S00003 HSC
40 S00003 PLT
41 N00039 NK
41 N00039 CD8
41 N00039 ncMono
41 N00039 cMono
41 N00039 CD4
41 N00039 B
41 N00039 MAIT
41 N00039 PB
41 N00039 DN
41 N00039 DP
41 N00039 DC
41 N00039 GDT
41 N00039 HSC
42 H00058 NK
42 H00058 CD8
42 H00058 ncMono
42 H00058 cMono
42 H00058 CD4
42 H00058 B
42 H00058 MAIT
42 H00058 PB
42 H00058 iNKT
42 H00058 DN
42 H00058 DP
42 H00058 DC
42 H00058 GDT
42 H00058 HSC
42 H00058 PLT
43 S00064 NK
43 S00064 CD8
43 S00064 ncMono
43 S00064 cMono
43 S00064 CD4
43 S00064 B
43 S00064 MAIT
43 S00064 PB
43 S00064 iNKT
43 S00064 DN
43 S00064 DP
43 S00064 DC
43 S00064 GDT
43 S00064 HSC
43 S00064 PLT
44 N00047 NK
44 N00047 CD8
44 N00047 ncMono
44 N00047 cMono
44 N00047 CD4
44 N00047 B
44 N00047 MAIT
44 N00047 PB
44 N00047 DN
44 N00047 DP
44 N00047 DC
44 N00047 GDT
44 N00047 HSC
44 N0004

80 N00037 NK
80 N00037 CD8
80 N00037 ncMono
80 N00037 cMono
80 N00037 CD4
80 N00037 B
80 N00037 MAIT
80 N00037 PB
80 N00037 DN
80 N00037 DP
80 N00037 DC
80 N00037 GDT
80 N00037 HSC
80 N00037 PLT
80 N00037 RET
80 N00037 Mast
81 S00002 NK
81 S00002 CD8
81 S00002 ncMono
81 S00002 cMono
81 S00002 CD4
81 S00002 B
81 S00002 MAIT
81 S00002 PB
81 S00002 DN
81 S00002 DP
81 S00002 DC
81 S00002 GDT
81 S00002 HSC
82 S00076 NK
82 S00076 CD8
82 S00076 ncMono
82 S00076 cMono
82 S00076 CD4
82 S00076 B
82 S00076 MAIT
82 S00076 PB
82 S00076 iNKT
82 S00076 DN
82 S00076 DP
82 S00076 DC
82 S00076 GDT
82 S00076 HSC
82 S00076 PLT
82 S00076 Mast
83 S00072 NK
83 S00072 CD8
83 S00072 ncMono
83 S00072 cMono
83 S00072 CD4
83 S00072 B
83 S00072 MAIT
83 S00072 PB
83 S00072 iNKT
83 S00072 DN
83 S00072 DP
83 S00072 DC
83 S00072 GDT
83 S00072 HSC
83 S00072 PLT
84 N00038 NK
84 N00038 CD8
84 N00038 ncMono
84 N00038 cMono
84 N00038 CD4
84 N00038 B
84 N00038 MAIT
84 N00038 PB
84 N00038 DN
84 N00038 DP
84 N00038 DC
84 N000

## Write adt_cite pseudobulk dict

In [46]:
with open('adt_cite_pseudobulks.pickle', 'wb') as f:
    pickle.dump(adt_cite_pseudobulks, f)

## Load adt_cite pseudobulk dict

In [47]:
with open("adt_cite_pseudobulks.pickle", "rb") as f:
    adt_cite_pseudobulks = pickle.load(f)

In [48]:
adt_cite_pseudobulks.keys()

dict_keys(['NK', 'CD8', 'ncMono', 'cMono', 'CD4', 'B', 'MAIT', 'PB', 'iNKT', 'DN', 'DP', 'DC', 'GDT', 'HSC', 'PLT', 'RET', 'Mast'])

In [49]:
adt_cite_pseudobulks['NK']

,CD80,CD86,CD274_B7_H1_PD_L1,CD273_B7_DC_PD_L2,CD275_B7_H2_ICOSL,humanCD11b,CD252_OX40L,CD137L_4_1BBLigand,CD155_PVR,CD112_Nectin_2,...,CD101_BB27,CD360_IL_21R,CD88_C5aR,HLA_F,NLRP2,Podocalyxin,CD224,c_Met,CD258_LIGHT,DR3_TRAMP
S00109,9160.0,893.0,5196.0,8835.0,9288.0,160386.0,3928.0,11486.0,9165.0,7158.0,...,19023.0,20263.0,14786.0,12916.0,26979.0,4190.0,16008.0,11854.0,9046.0,18542.0
S00112,1616.0,260.0,714.0,1209.0,1553.0,29489.0,632.0,1755.0,1771.0,1189.0,...,3173.0,2969.0,2568.0,1903.0,4507.0,800.0,2835.0,2156.0,1285.0,2716.0
S00005,2155.0,359.0,1103.0,1474.0,2103.0,36432.0,875.0,2589.0,2408.0,1705.0,...,4894.0,4260.0,3500.0,2699.0,6510.0,1098.0,6534.0,3030.0,1705.0,3529.0
S00061,1067.0,103.0,531.0,673.0,1006.0,20431.0,410.0,1145.0,1260.0,780.0,...,2054.0,1751.0,1454.0,1196.0,2973.0,520.0,5033.0,1392.0,836.0,1554.0
S00056,2572.0,249.0,1322.0,2053.0,2542.0,49007.0,995.0,2996.0,2417.0,1884.0,...,4808.0,5319.0,3881.0,3281.0,7196.0,1219.0,4005.0,3329.0,2394.0,5122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N00037,1377.0,128.0,696.0,1081.0,1381.0,21662.0,572.0,1771.0,1192.0,1014.0,...,2592.0,2824.0,2881.0,1852.0,4365.0,698.0,2955.0,1778.0,1189.0,2280.0
S00002,408.0,47.0,197.0,297.0,419.0,7930.0,163.0,535.0,497.0,311.0,...,771.0,649.0,678.0,634.0,1486.0,223.0,971.0,510.0,318.0,713.0
S00076,1868.0,147.0,843.0,1086.0,2043.0,37787.0,813.0,2348.0,2071.0,1501.0,...,3086.0,3757.0,2884.0,2737.0,6283.0,953.0,3240.0,2454.0,1689.0,3066.0
S00072,2238.0,184.0,1136.0,1308.0,2273.0,39893.0,891.0,2734.0,2322.0,1811.0,...,3548.0,4283.0,3370.0,3188.0,7534.0,1175.0,3291.0,2962.0,1954.0,3493.0


## Create cytof pseudobulks

In [50]:
cytof_pseudobulks = {}
for i, patient in enumerate(pd.unique(cytof.obs.patient_id)):
    cytof_patient = cytof[cytof.obs.patient_id == patient, :]
    for cell_type in pd.unique(cytof.obs.Annotation_major_subset):
        cytof_patient_cell_type = cytof_patient[cytof_patient.obs.Annotation_major_subset == cell_type, :]
        if len(cytof_patient_cell_type) > 0:
            print(i, patient, cell_type)
            key = f'{cell_type}'
            row = cytof_patient_cell_type.X.toarray()
            if row.shape[0] > 1:
                row = row.mean(axis=0)
            else:
                row = row[0]
            if key not in cytof_pseudobulks:
                df = pd.DataFrame(columns=cytof.var_names)
                df.loc[patient] = row
                cytof_pseudobulks[key] = df
            else:
                cytof_pseudobulks[key].loc[patient] = row

            print(i, patient, cell_type)

0 H00067 CD8
0 H00067 CD8
0 H00067 B
0 H00067 B
0 H00067 cMono
0 H00067 cMono
0 H00067 CD4
0 H00067 CD4
0 H00067 GDT
0 H00067 GDT
0 H00067 DC
0 H00067 DC
0 H00067 Basophil
0 H00067 Basophil
0 H00067 NK
0 H00067 NK
0 H00067 ncMono
0 H00067 ncMono
0 H00067 MAIT
0 H00067 MAIT
0 H00067 DN
0 H00067 DN
0 H00067 PB
0 H00067 PB
1 N00023 CD8
1 N00023 CD8
1 N00023 B
1 N00023 B
1 N00023 cMono
1 N00023 cMono
1 N00023 CD4
1 N00023 CD4
1 N00023 GDT
1 N00023 GDT
1 N00023 DC
1 N00023 DC
1 N00023 Basophil
1 N00023 Basophil
1 N00023 NK
1 N00023 NK
1 N00023 ncMono
1 N00023 ncMono
1 N00023 MAIT
1 N00023 MAIT
1 N00023 DN
1 N00023 DN
1 N00023 PB
1 N00023 PB
2 S00028 CD8
2 S00028 CD8
2 S00028 B
2 S00028 B
2 S00028 cMono
2 S00028 cMono
2 S00028 CD4
2 S00028 CD4
2 S00028 GDT
2 S00028 GDT
2 S00028 DC
2 S00028 DC
2 S00028 Basophil
2 S00028 Basophil
2 S00028 NK
2 S00028 NK
2 S00028 ncMono
2 S00028 ncMono
2 S00028 MAIT
2 S00028 MAIT
2 S00028 DN
2 S00028 DN
2 S00028 PB
2 S00028 PB
3 G05064 CD8
3 G05064 CD8
3 G05064

25 S00039 CD8
25 S00039 CD8
25 S00039 B
25 S00039 B
25 S00039 cMono
25 S00039 cMono
25 S00039 CD4
25 S00039 CD4
25 S00039 GDT
25 S00039 GDT
25 S00039 DC
25 S00039 DC
25 S00039 Basophil
25 S00039 Basophil
25 S00039 NK
25 S00039 NK
25 S00039 ncMono
25 S00039 ncMono
25 S00039 MAIT
25 S00039 MAIT
25 S00039 DN
25 S00039 DN
25 S00039 PB
25 S00039 PB
26 S00016 CD8
26 S00016 CD8
26 S00016 B
26 S00016 B
26 S00016 cMono
26 S00016 cMono
26 S00016 CD4
26 S00016 CD4
26 S00016 GDT
26 S00016 GDT
26 S00016 DC
26 S00016 DC
26 S00016 Basophil
26 S00016 Basophil
26 S00016 NK
26 S00016 NK
26 S00016 ncMono
26 S00016 ncMono
26 S00016 MAIT
26 S00016 MAIT
26 S00016 DN
26 S00016 DN
26 S00016 PB
26 S00016 PB
27 S00129 CD8
27 S00129 CD8
27 S00129 B
27 S00129 B
27 S00129 cMono
27 S00129 cMono
27 S00129 CD4
27 S00129 CD4
27 S00129 GDT
27 S00129 GDT
27 S00129 DC
27 S00129 DC
27 S00129 Basophil
27 S00129 Basophil
27 S00129 NK
27 S00129 NK
27 S00129 ncMono
27 S00129 ncMono
27 S00129 MAIT
27 S00129 MAIT
27 S00129 DN
2

49 S00113 CD8
49 S00113 CD8
49 S00113 B
49 S00113 B
49 S00113 cMono
49 S00113 cMono
49 S00113 CD4
49 S00113 CD4
49 S00113 GDT
49 S00113 GDT
49 S00113 DC
49 S00113 DC
49 S00113 Basophil
49 S00113 Basophil
49 S00113 NK
49 S00113 NK
49 S00113 ncMono
49 S00113 ncMono
49 S00113 MAIT
49 S00113 MAIT
49 S00113 DN
49 S00113 DN
49 S00113 PB
49 S00113 PB
50 S00134 CD8
50 S00134 CD8
50 S00134 B
50 S00134 B
50 S00134 cMono
50 S00134 cMono
50 S00134 CD4
50 S00134 CD4
50 S00134 GDT
50 S00134 GDT
50 S00134 DC
50 S00134 DC
50 S00134 Basophil
50 S00134 Basophil
50 S00134 NK
50 S00134 NK
50 S00134 ncMono
50 S00134 ncMono
50 S00134 MAIT
50 S00134 MAIT
50 S00134 DN
50 S00134 DN
50 S00134 PB
50 S00134 PB
51 S00148 CD8
51 S00148 CD8
51 S00148 B
51 S00148 B
51 S00148 cMono
51 S00148 cMono
51 S00148 CD4
51 S00148 CD4
51 S00148 GDT
51 S00148 GDT
51 S00148 DC
51 S00148 DC
51 S00148 Basophil
51 S00148 Basophil
51 S00148 NK
51 S00148 NK
51 S00148 ncMono
51 S00148 ncMono
51 S00148 MAIT
51 S00148 MAIT
51 S00148 DN
5

73 H00064 CD8
73 H00064 CD8
73 H00064 B
73 H00064 B
73 H00064 cMono
73 H00064 cMono
73 H00064 CD4
73 H00064 CD4
73 H00064 GDT
73 H00064 GDT
73 H00064 DC
73 H00064 DC
73 H00064 Basophil
73 H00064 Basophil
73 H00064 NK
73 H00064 NK
73 H00064 ncMono
73 H00064 ncMono
73 H00064 MAIT
73 H00064 MAIT
73 H00064 DN
73 H00064 DN
73 H00064 PB
73 H00064 PB
74 S00002 CD8
74 S00002 CD8
74 S00002 B
74 S00002 B
74 S00002 cMono
74 S00002 cMono
74 S00002 CD4
74 S00002 CD4
74 S00002 GDT
74 S00002 GDT
74 S00002 DC
74 S00002 DC
74 S00002 Basophil
74 S00002 Basophil
74 S00002 NK
74 S00002 NK
74 S00002 ncMono
74 S00002 ncMono
74 S00002 MAIT
74 S00002 MAIT
74 S00002 DN
74 S00002 DN
74 S00002 PB
74 S00002 PB
75 S00082 CD8
75 S00082 CD8
75 S00082 B
75 S00082 B
75 S00082 cMono
75 S00082 cMono
75 S00082 CD4
75 S00082 CD4
75 S00082 GDT
75 S00082 GDT
75 S00082 DC
75 S00082 DC
75 S00082 Basophil
75 S00082 Basophil
75 S00082 NK
75 S00082 NK
75 S00082 ncMono
75 S00082 ncMono
75 S00082 MAIT
75 S00082 MAIT
75 S00082 DN
7

## Write cytof pseudobulk dict

In [51]:
with open('cytof_pseudobulks.pickle', 'wb') as f:
    pickle.dump(cytof_pseudobulks, f)

## Load cytof pseudobulk dict

In [52]:
with open("cytof_pseudobulks.pickle", "rb") as f:
    cytof_pseudobulks = pickle.load(f)

In [53]:
cytof_pseudobulks.keys()

dict_keys(['CD8', 'B', 'cMono', 'CD4', 'GDT', 'DC', 'Basophil', 'NK', 'ncMono', 'MAIT', 'DN', 'PB'])

In [54]:
cytof_pseudobulks['PB']

,CD16,CD19,CD3,IgG,CD4,HLA_DR,CTLA4,Siglec_8,CD28,Ki_67,...,KLGR1,FOXP3,CD38,CD45,CD123,CD25,CD141,CLA,CX3CR1,Event_length
H00067,0.256622,1.793438,0.164827,0.690555,0.080794,2.886634,0.449791,0.346964,0.569146,0.610600,...,0.221222,1.750779,4.769200,4.736462,0.493594,0.636633,0.309722,0.479905,0.170055,2.362324
N00023,0.234235,1.399907,0.356940,0.517184,0.230798,4.023304,0.437576,0.286060,0.564202,0.594795,...,0.262866,1.580741,5.752856,4.506413,0.570322,0.500614,0.293156,0.727019,0.220616,2.335442
S00028,0.179987,1.109812,0.219208,0.773337,0.124894,3.623053,0.485388,0.229833,0.682051,0.911767,...,0.186357,1.448861,5.532031,4.566329,0.451379,0.382546,0.306191,0.619152,0.197398,2.376737
G05064,0.209952,1.188672,0.179760,0.703398,0.139180,4.226169,0.425401,0.299730,0.547411,0.333854,...,0.117253,1.305063,5.881768,4.899231,0.246277,0.469947,0.451339,0.732454,0.182269,2.513179
S00112,0.197682,1.317763,0.263495,1.058457,0.127730,3.714065,0.409244,0.230303,0.697230,0.563165,...,0.190732,1.643513,6.313135,4.554265,0.507629,0.494924,0.332050,0.895867,0.163344,2.404076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S00024,0.236750,0.670671,0.271233,0.833529,0.223617,3.683098,0.530697,0.239618,0.623732,0.557322,...,0.219388,0.963160,5.319296,4.663140,0.545138,0.307208,0.250450,1.035268,0.188867,2.298638
N00027,0.230693,1.475566,0.351764,1.042030,0.242289,3.779344,0.325567,0.170726,0.475141,0.326478,...,0.204069,1.141675,5.045805,4.528763,0.382801,0.433258,0.211424,0.588969,0.229494,2.296694
S00066,0.258156,0.863628,0.224959,0.623558,0.171820,3.026447,0.367692,0.157621,0.561173,1.013820,...,0.153432,1.127147,5.560332,4.616500,0.298200,0.309126,0.210060,0.723372,0.099695,2.385729
S00096,0.292538,1.090976,0.364000,0.546217,0.297926,3.989407,0.461439,0.197102,0.520210,1.214241,...,0.178490,1.155260,5.830649,4.583470,0.459285,0.288856,0.215363,0.905374,0.157534,2.371234


## Generate rna_bulk pseudobulks

In [55]:
rna_bulk.var_names.isin(gene_membership_bulk.gene_id)

array([ True, False,  True, ...,  True, False,  True])

In [56]:
rna_bulk = rna_bulk[:,rna_bulk.var_names.isin(gene_membership_bulk.gene_id)]

In [57]:
rna_bulk.var_names.isin(gene_membership_bulk.gene_id).all()

True

In [58]:
gene_membership_bulk.gene_id.isin(rna_bulk.var_names).all()

True

In [59]:
rna_bulk.var['id'] = rna_bulk.var_names.copy()
rna_bulk.var

/tmp/ipykernel_1827733/4254468636.py:1: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  rna_bulk.var['id'] = rna_bulk.var_names.copy()


,id
ENSG00000000003,ENSG00000000003
ENSG00000000419,ENSG00000000419
ENSG00000000457,ENSG00000000457
ENSG00000000460,ENSG00000000460
ENSG00000000938,ENSG00000000938
...,...
ENSG00000288597,ENSG00000288597
ENSG00000288598,ENSG00000288598
ENSG00000288600,ENSG00000288600
ENSG00000288602,ENSG00000288602


In [60]:
gene_name_map = {}
for i, gene_id in enumerate(gene_membership_bulk.gene_id):
    if gene_id not in gene_name_map:
        gene_name_map[gene_id] = gene_membership_bulk.gene_name.iloc[i].strip()
    elif gene_membership_bulk.gene_name.iloc[i] != gene_name_map[gene_id]:
        raise ValueError(f'ID to name is not a 1:1 map from ID {gene_id}')
gene_name_map

{'ENSG00000105173': 'CCNE1',
 'ENSG00000101003': 'GINS1',
 'ENSG00000168496': 'FEN1',
 'ENSG00000123352': 'SPATS2',
 'ENSG00000278588': 'H2BC10',
 'ENSG00000112312': 'GMNN',
 'ENSG00000169679': 'BUB1',
 'ENSG00000149554': 'CHEK1',
 'ENSG00000077152': 'UBE2T',
 'ENSG00000183010': 'PYCR1',
 'ENSG00000071539': 'TRIP13',
 'ENSG00000048462': 'TNFRSF17',
 'ENSG00000171320': 'ESCO2',
 'ENSG00000135916': 'ITM2C',
 'ENSG00000074842': 'MYDGF',
 'ENSG00000101057': 'MYBL2',
 'ENSG00000239672': 'NME1',
 'ENSG00000065328': 'MCM10',
 'ENSG00000104738': 'MCM4',
 'ENSG00000122952': 'ZWINT',
 'ENSG00000166562': 'SEC11C',
 'ENSG00000115902': 'SLC1A4',
 'ENSG00000117411': 'B4GALT2',
 'ENSG00000174371': 'EXO1',
 'ENSG00000123989': 'CHPF',
 'ENSG00000170476': 'MZB1',
 'ENSG00000275714': 'H3C1',
 'ENSG00000182481': 'KPNA2',
 'ENSG00000155660': 'PDIA4',
 'ENSG00000065308': 'TRAM2',
 'ENSG00000143476': 'DTL',
 'ENSG00000166598': 'HSP90B1',
 'ENSG00000168078': 'PBK',
 'ENSG00000108829': 'LRRC59',
 'ENSG00000137

In [61]:
rna_bulk.var_names = rna_bulk.var.id.map(gene_name_map)
rna_bulk.var_names

Index(['TSPAN6', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2', 'GCLC',
       'NFYA', 'STPG1',
       ...
       'AC098872.2', 'AL357874.3', 'AP001054.1', 'AP002800.2', 'C8orf44',
       'AC234782.4', 'AL354833.1', 'AL354833.2', 'C8orf44-SGK3', 'AC015871.8'],
      dtype='object', name='id', length=23063)

In [62]:
gene_membership_bulk

,module,gene_id,gene_name,membership,p.value
0,greenyellow,ENSG00000105173,CCNE1,0.958784,6.194695e-79
1,greenyellow,ENSG00000101003,GINS1,0.954818,3.500631e-76
2,greenyellow,ENSG00000168496,FEN1,0.953113,4.493982e-75
3,greenyellow,ENSG00000123352,SPATS2,0.949671,5.878363e-73
4,greenyellow,ENSG00000278588,H2BC10,0.946806,2.635032e-71
...,...,...,...,...,...
23058,grey,ENSG00000251996,AC007284.1,0.800786,3.415287e-33
23059,grey,ENSG00000226918,AC010086.1,0.779957,1.710957e-30
23060,grey,ENSG00000272787,AC253536.6,-0.169222,4.333857e-02
23061,grey,ENSG00000264057,AC103810.1,0.089963,2.852850e-01


In [63]:
rna_bulk.obs

,patient_id
0,S00016
1,S00020
2,S00024
3,S00027
4,S00028
...,...
137,S00095
138,S00096
140,S00099
141,S00104


In [64]:
rna_bulk.obs.index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '130', '132', '133', '134', '135', '137', '138', '140', '141', '142'],
      dtype='object', length=103)

In [65]:
gene_membership_bulk

,module,gene_id,gene_name,membership,p.value
0,greenyellow,ENSG00000105173,CCNE1,0.958784,6.194695e-79
1,greenyellow,ENSG00000101003,GINS1,0.954818,3.500631e-76
2,greenyellow,ENSG00000168496,FEN1,0.953113,4.493982e-75
3,greenyellow,ENSG00000123352,SPATS2,0.949671,5.878363e-73
4,greenyellow,ENSG00000278588,H2BC10,0.946806,2.635032e-71
...,...,...,...,...,...
23058,grey,ENSG00000251996,AC007284.1,0.800786,3.415287e-33
23059,grey,ENSG00000226918,AC010086.1,0.779957,1.710957e-30
23060,grey,ENSG00000272787,AC253536.6,-0.169222,4.333857e-02
23061,grey,ENSG00000264057,AC103810.1,0.089963,2.852850e-01


In [66]:
#redundant_gene_names_bulk = [gene_name for gene_name in pd.unique(gene_membership_bulk.gene_name) if len(np.where(gene_membership_bulk.gene_name == gene_name)[0]) > 1 ]
#redundant_gene_names_bulk

In [67]:
#gene_membership_bulk.iloc[np.where(gene_membership_bulk.gene_name == 'TBCE')[0]]

In [68]:
#genes_to_drop = np.where(pd.Series(gene_membership_bulk.gene_name).isin(redundant_gene_names_bulk))[0]
#genes_to_drop

In [69]:
#gene_membership_bulk.drop(genes_to_drop, axis=0, inplace=True)

In [70]:
#redundant_gene_names_bulk = [gene_name for gene_name in pd.unique(gene_membership_bulk.gene_name) if len(np.where(gene_membership_bulk.gene_name == gene_name)[0]) > 1 ]
#redundant_gene_names_bulk

In [71]:
rna_bulk_pseudobulks = {}
for i, patient in enumerate(pd.unique(rna_bulk.obs.patient_id)):
    rna_bulk_patient = rna_bulk[rna_bulk.obs.patient_id == patient, :]
    for module in pd.unique(gene_membership_bulk.module):
        print(i, patient, module)
        gene_membership_bulk_module = gene_membership_bulk.loc[gene_membership_bulk.module == module, 'gene_name']
        rna_bulk_patient_module = rna_bulk_patient[:, gene_membership_bulk_module.index]
        key = f'{module}'
        row = rna_bulk_patient_module.X.toarray()
        if row.shape[0] > 1:
            row = row.sum(axis=0)
        else:
            row = row[0]
        if key not in rna_bulk_pseudobulks:
            df = pd.DataFrame(index = pd.unique(rna_bulk.obs.patient_id), columns = gene_membership_bulk_module)
            df.iloc[i] = row
            rna_bulk_pseudobulks[key] = df
        else:
            rna_bulk_pseudobulks[key].iloc[i] = row

0 S00016 greenyellow
0 S00016 green
0 S00016 magenta
0 S00016 lightgreen
0 S00016 black
0 S00016 turquoise
0 S00016 lightcyan
0 S00016 midnightblue
0 S00016 blue
0 S00016 grey60
0 S00016 purple
0 S00016 cyan
0 S00016 grey
1 S00020 greenyellow
1 S00020 green
1 S00020 magenta
1 S00020 lightgreen
1 S00020 black
1 S00020 turquoise
1 S00020 lightcyan
1 S00020 midnightblue
1 S00020 blue
1 S00020 grey60
1 S00020 purple
1 S00020 cyan
1 S00020 grey
2 S00024 greenyellow
2 S00024 green
2 S00024 magenta
2 S00024 lightgreen
2 S00024 black
2 S00024 turquoise
2 S00024 lightcyan
2 S00024 midnightblue
2 S00024 blue
2 S00024 grey60
2 S00024 purple
2 S00024 cyan
2 S00024 grey
3 S00027 greenyellow
3 S00027 green
3 S00027 magenta
3 S00027 lightgreen
3 S00027 black
3 S00027 turquoise
3 S00027 lightcyan
3 S00027 midnightblue
3 S00027 blue
3 S00027 grey60
3 S00027 purple
3 S00027 cyan
3 S00027 grey
4 S00028 greenyellow
4 S00028 green
4 S00028 magenta
4 S00028 lightgreen
4 S00028 black
4 S00028 turquoise
4 S00

37 N00006 cyan
37 N00006 grey
38 N00007 greenyellow
38 N00007 green
38 N00007 magenta
38 N00007 lightgreen
38 N00007 black
38 N00007 turquoise
38 N00007 lightcyan
38 N00007 midnightblue
38 N00007 blue
38 N00007 grey60
38 N00007 purple
38 N00007 cyan
38 N00007 grey
39 N00017 greenyellow
39 N00017 green
39 N00017 magenta
39 N00017 lightgreen
39 N00017 black
39 N00017 turquoise
39 N00017 lightcyan
39 N00017 midnightblue
39 N00017 blue
39 N00017 grey60
39 N00017 purple
39 N00017 cyan
39 N00017 grey
40 N00018 greenyellow
40 N00018 green
40 N00018 magenta
40 N00018 lightgreen
40 N00018 black
40 N00018 turquoise
40 N00018 lightcyan
40 N00018 midnightblue
40 N00018 blue
40 N00018 grey60
40 N00018 purple
40 N00018 cyan
40 N00018 grey
41 N00021 greenyellow
41 N00021 green
41 N00021 magenta
41 N00021 lightgreen
41 N00021 black
41 N00021 turquoise
41 N00021 lightcyan
41 N00021 midnightblue
41 N00021 blue
41 N00021 grey60
41 N00021 purple
41 N00021 cyan
41 N00021 grey
42 N00024 greenyellow
42 N0002

77 S00078 purple
77 S00078 cyan
77 S00078 grey
78 S00081 greenyellow
78 S00081 green
78 S00081 magenta
78 S00081 lightgreen
78 S00081 black
78 S00081 turquoise
78 S00081 lightcyan
78 S00081 midnightblue
78 S00081 blue
78 S00081 grey60
78 S00081 purple
78 S00081 cyan
78 S00081 grey
79 S00082 greenyellow
79 S00082 green
79 S00082 magenta
79 S00082 lightgreen
79 S00082 black
79 S00082 turquoise
79 S00082 lightcyan
79 S00082 midnightblue
79 S00082 blue
79 S00082 grey60
79 S00082 purple
79 S00082 cyan
79 S00082 grey
80 S00095 greenyellow
80 S00095 green
80 S00095 magenta
80 S00095 lightgreen
80 S00095 black
80 S00095 turquoise
80 S00095 lightcyan
80 S00095 midnightblue
80 S00095 blue
80 S00095 grey60
80 S00095 purple
80 S00095 cyan
80 S00095 grey
81 S00096 greenyellow
81 S00096 green
81 S00096 magenta
81 S00096 lightgreen
81 S00096 black
81 S00096 turquoise
81 S00096 lightcyan
81 S00096 midnightblue
81 S00096 blue
81 S00096 grey60
81 S00096 purple
81 S00096 cyan
81 S00096 grey
82 S00099 gre

In [72]:
rna_bulk_pseudobulks['greenyellow']

gene_name,CCNE1,GINS1,FEN1,SPATS2,H2BC10,GMNN,BUB1,CHEK1,UBE2T,PYCR1,...,AC027601.1,DYRK4,SNX17,ESRRB,XAB2,CTBP1-AS,UBE2Z,Z95118.2,AC239800.1,GFOD1
S00016,14.0,708.0,1203.0,293.0,31102.0,408.0,847.0,1264.0,2549.0,107.0,...,22292.0,262.0,2334.0,1962.0,3776.0,12982.0,442.0,7.0,1027.0,8851.0
S00020,14.0,629.0,787.0,430.0,22334.0,115.0,422.0,2710.0,2028.0,72.0,...,13829.0,173.0,1498.0,1630.0,2109.0,8965.0,966.0,5.0,2777.0,7173.0
S00024,6.0,734.0,943.0,280.0,34538.0,126.0,1106.0,823.0,3229.0,57.0,...,30738.0,195.0,1982.0,2377.0,2886.0,7860.0,390.0,3.0,814.0,6965.0
S00027,0.0,488.0,820.0,135.0,37673.0,37.0,393.0,304.0,2835.0,30.0,...,26831.0,91.0,697.0,670.0,1298.0,13398.0,506.0,15.0,1586.0,6529.0
S00028,20.0,1249.0,1363.0,502.0,46265.0,171.0,2193.0,1566.0,3590.0,127.0,...,58114.0,464.0,3503.0,3131.0,4608.0,13903.0,644.0,15.0,989.0,14260.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S00095,8.0,513.0,847.0,277.0,28571.0,56.0,461.0,976.0,2201.0,68.0,...,18861.0,93.0,894.0,954.0,1649.0,10364.0,508.0,14.0,2499.0,8134.0
S00096,4.0,522.0,1242.0,215.0,33844.0,217.0,483.0,670.0,2665.0,24.0,...,19771.0,110.0,803.0,707.0,1259.0,12643.0,273.0,7.0,583.0,5752.0
S00099,12.0,751.0,942.0,344.0,41686.0,90.0,918.0,2548.0,2755.0,122.0,...,29020.0,195.0,1672.0,1676.0,2448.0,12419.0,659.0,7.0,1255.0,8770.0
S00104,28.0,1017.0,1148.0,283.0,33638.0,216.0,1132.0,1190.0,2655.0,123.0,...,36334.0,297.0,3155.0,3159.0,4178.0,10633.0,718.0,4.0,1457.0,10513.0


In [73]:
rna_bulk

AnnData object with n_obs × n_vars = 103 × 23063
    obs: 'patient_id'
    var: 'id'

## Write rna_bulk pseudobulk dict

In [74]:
with open('rna_bulk_pseudobulks.pickle', 'wb') as f:
    pickle.dump(rna_bulk_pseudobulks, f)

## Load rna_bulk pseudobulk dict

In [75]:
with open("rna_bulk_pseudobulks.pickle", "rb") as f:
    rna_bulk_pseudobulks = pickle.load(f)

In [76]:
rna_bulk_pseudobulks['greenyellow']

gene_name,CCNE1,GINS1,FEN1,SPATS2,H2BC10,GMNN,BUB1,CHEK1,UBE2T,PYCR1,...,AC027601.1,DYRK4,SNX17,ESRRB,XAB2,CTBP1-AS,UBE2Z,Z95118.2,AC239800.1,GFOD1
S00016,14.0,708.0,1203.0,293.0,31102.0,408.0,847.0,1264.0,2549.0,107.0,...,22292.0,262.0,2334.0,1962.0,3776.0,12982.0,442.0,7.0,1027.0,8851.0
S00020,14.0,629.0,787.0,430.0,22334.0,115.0,422.0,2710.0,2028.0,72.0,...,13829.0,173.0,1498.0,1630.0,2109.0,8965.0,966.0,5.0,2777.0,7173.0
S00024,6.0,734.0,943.0,280.0,34538.0,126.0,1106.0,823.0,3229.0,57.0,...,30738.0,195.0,1982.0,2377.0,2886.0,7860.0,390.0,3.0,814.0,6965.0
S00027,0.0,488.0,820.0,135.0,37673.0,37.0,393.0,304.0,2835.0,30.0,...,26831.0,91.0,697.0,670.0,1298.0,13398.0,506.0,15.0,1586.0,6529.0
S00028,20.0,1249.0,1363.0,502.0,46265.0,171.0,2193.0,1566.0,3590.0,127.0,...,58114.0,464.0,3503.0,3131.0,4608.0,13903.0,644.0,15.0,989.0,14260.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S00095,8.0,513.0,847.0,277.0,28571.0,56.0,461.0,976.0,2201.0,68.0,...,18861.0,93.0,894.0,954.0,1649.0,10364.0,508.0,14.0,2499.0,8134.0
S00096,4.0,522.0,1242.0,215.0,33844.0,217.0,483.0,670.0,2665.0,24.0,...,19771.0,110.0,803.0,707.0,1259.0,12643.0,273.0,7.0,583.0,5752.0
S00099,12.0,751.0,942.0,344.0,41686.0,90.0,918.0,2548.0,2755.0,122.0,...,29020.0,195.0,1672.0,1676.0,2448.0,12419.0,659.0,7.0,1255.0,8770.0
S00104,28.0,1017.0,1148.0,283.0,33638.0,216.0,1132.0,1190.0,2655.0,123.0,...,36334.0,297.0,3155.0,3159.0,4178.0,10633.0,718.0,4.0,1457.0,10513.0


## Generate luminex pseudobulk

In [77]:
redundant_luminex_patients = pd.Series([patient for patient in pd.unique(luminex.obs.patient_id) if (luminex.obs.patient_id == patient).sum() > 1])
len(redundant_luminex_patients)

31

In [78]:
np.where(luminex.obs.patient_id.isin(redundant_luminex_patients))

(array([  0,   1,   3,   4,   5,   8,   9,  11,  13,  15,  17,  18,  19,
         21,  22,  23,  24,  27,  28,  29,  30,  31,  32,  33,  34,  35,
         36,  37,  38,  39,  40,  41,  42,  43,  44,  47,  48,  52,  53,
         55,  56,  57,  59,  61,  62,  63,  66,  67,  70,  71,  72,  73,
         74,  79,  87,  88,  93,  94,  95,  96, 102, 103, 104, 105, 109,
        111]),)

In [79]:
[luminex.obs.patient_id.loc[luminex.obs.patient_id == redundant_luminex_patients[i]] for i in range(len(redundant_luminex_patients))]

[2     S00052
 32    S00052
 Name: patient_id, dtype: object,
 3     S00109
 35    S00109
 Name: patient_id, dtype: object,
 5    S00027
 6    S00027
 Name: patient_id, dtype: object,
 8     S00124
 63    S00124
 Name: patient_id, dtype: object,
 11    S00007
 12    S00007
 Name: patient_id, dtype: object,
 16    S00043
 18    S00043
 Name: patient_id, dtype: object,
 23    S00020
 76    S00020
 Name: patient_id, dtype: object,
 25    S00005
 81    S00005
 Name: patient_id, dtype: object,
 26    S00050
 27    S00050
 Name: patient_id, dtype: object,
 33    S00148
 34    S00148
 Name: patient_id, dtype: object,
 39    S00061
 56    S00061
 Name: patient_id, dtype: object,
 40    S00033
 41    S00033
 42    S00033
 Name: patient_id, dtype: object,
 43    S00081
 44    S00081
 Name: patient_id, dtype: object,
 47    S00048
 48    S00048
 Name: patient_id, dtype: object,
 49    S00041
 59    S00041
 60    S00041
 Name: patient_id, dtype: object,
 50    S00045
 51    S00045
 52    S00045
 N

In [80]:
redundant_rna_bulk_patients = pd.Series([patient for patient in pd.unique(rna_bulk.obs.patient_id) if (rna_bulk.obs.patient_id == patient).sum() > 1])
len(redundant_rna_bulk_patients)

15

In [81]:
np.where(rna_bulk.obs.patient_id.isin(redundant_rna_bulk_patients))

(array([ 6,  7,  9, 13, 14, 16, 17, 18, 19, 20, 31, 33, 34, 35, 44, 45, 50,
        51, 57, 58, 60, 61, 62, 68, 69, 70, 71, 76, 77, 80, 81, 95, 96]),)

In [82]:
[rna_bulk.obs.patient_id.loc[rna_bulk.obs.patient_id == redundant_rna_bulk_patients[i]] for i in range(len(redundant_rna_bulk_patients))]

[6    S00033
 7    S00033
 9    S00033
 Name: patient_id, dtype: object,
 14    S00042
 15    S00042
 Name: patient_id, dtype: object,
 17    S00045
 18    S00045
 19    S00045
 Name: patient_id, dtype: object,
 20    S00048
 21    S00048
 Name: patient_id, dtype: object,
 34    N00023
 72    N00023
 Name: patient_id, dtype: object,
 37    N00028
 55    N00028
 Name: patient_id, dtype: object,
 38    N00029
 39    N00029
 Name: patient_id, dtype: object,
 58    N00006
 69    N00006
 Name: patient_id, dtype: object,
 89    S00005
 90    S00005
 Name: patient_id, dtype: object,
 93    S00109
 94    S00109
 95    S00109
 Name: patient_id, dtype: object,
 101    S00124
 103    S00124
 Name: patient_id, dtype: object,
 104    S00129
 105    S00129
 Name: patient_id, dtype: object,
 111    S00052
 112    S00052
 Name: patient_id, dtype: object,
 115    S00056
 116    S00056
 Name: patient_id, dtype: object,
 133    S00081
 134    S00081
 Name: patient_id, dtype: object]

In [83]:
len(rna_bulk.obs.patient_id) - len(pd.unique(rna_bulk.obs.patient_id))

18

In [84]:
len(luminex.obs.patient_id) - len(pd.unique(luminex.obs.patient_id))

35

In [91]:
luminex_pseudobulks = pd.DataFrame(columns = luminex.var_names)
luminex_matrix = np.empty((0, len(luminex.var_names)))
for i, patient in enumerate(pd.unique(luminex.obs.patient_id)):
    print(f"{i} {patient}")
    luminex_patient = luminex[luminex.obs.patient_id==patient, :]
    row = luminex_patient.X.toarray()
    if row.shape[0] > 1:
        row = row.mean(axis=0)
    else:
        row = row[0]
    
    luminex_pseudobulks.loc[patient] = row

0 S00052
1 S00109
2 S00099
3 S00027
4 S00124
5 S00030
6 S00054
7 S00007
8 S00024
9 S00043
10 S00095
11 S00040
12 S00020
13 S00068
14 S00005
15 S00050
16 S00064
17 S00148
18 S00069
19 S00053
20 S00061
21 S00033
22 S00081
23 S00048
24 S00041
25 S00045
26 S00042
27 S00060
28 S00134
29 S00119
30 S00096
31 S00057
32 S00078
33 S00028
34 S00072
35 S00056
36 S00066
37 S00039
38 S00034
39 S00106
40 S00003
41 S00076
42 S00129
43 S00112
44 S00113
45 S00058
46 S00063
47 S00006
48 S00049
49 S00059
50 S00002
51 S00104
52 S00016
53 S00114
54 S00142
55 S00082
56 G05073
57 G05064
58 N00038
59 N00028
60 N00050
61 N00027
62 N00021
63 N00037
64 N00012
65 N00033
66 N00017
67 N00047
68 N00039
69 N00032
70 N00023
71 N00018
72 N00006
73 N00044
74 N00007
75 N00029
76 N00024
77 H00054
78 H00064
79 H00070
80 H00053
81 H00058
82 H00085
83 H00072
84 H00067


In [97]:
luminex.X[0,:]

ArrayView([7.6407977e+04, 4.5607180e+04, 5.1840969e+04, 8.8627438e+04,
           1.7456000e+02, 0.0000000e+00, 1.8459999e+01, 0.0000000e+00,
           0.0000000e+00, 2.0709999e+01, 0.0000000e+00, 0.0000000e+00,
           1.1300000e+00, 0.0000000e+00, 0.0000000e+00, 3.2650002e+01,
           0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
           1.7575999e+02, 1.9700000e+00, 1.0117700e+03, 8.7474297e+03,
           2.4151230e+04, 2.3000000e+00, 2.2064999e+02, 5.0999999e+00,
           1.9129999e+01, 1.4224001e+02, 1.3653999e+02, 0.0000000e+00,
           9.9923406e+05, 3.8549999e+01, 1.2294190e+04, 3.5385001e+02,
           1.4299999e+00, 2.0200000e+00, 0.0000000e+00, 0.0000000e+00,
           1.0910000e+01, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
           3.7400000e+00, 0.0000000e+00, 3.8000000e-01, 0.0000000e+00,
           2.3800001e+00, 0.0000000e+00, 2.6900000e+02], dtype=float32)

## Write luminex pseudobulks

In [98]:
luminex_pseudobulks.to_csv("luminex_pseudobulks.csv")

## load luminex pseudobulks

In [99]:
luminex_pseudobulks = pd.read_csv("luminex_pseudobulks.csv", index_col = 0)
luminex_pseudobulks

,CCL18/PARC (BR33) (33) low,Lactoferrin (BR36) (36) high,Lipocalin-2/NGAL (BR21) (21) high,Myeloperoxidase/MPO (BR53) (53) high,CCL2/JE/MCP-1 (BR25) (25) high,CCL3/MIP-1 alpha (BR35) (35) high,CCL4/MIP-1 beta (BR37) (37) high,CCL17/TARC (BR75) (75) high,CXCL1/GRO alpha/KC/CINC-1 (BR77) (77) high,CXCL5/ENA-78 (BR52) (52) high,...,IFN-alpha (BR63) (63) high,IL-2 (BR43) (43) high,IL-5 (BR53) (53) high,IL-8/CXCL8 (BR48) (48) high,IL-12 p70 (BR56) (56) high,IL-15 (BR52) (52) high,IL-23 (BR76) (76) high,IL-33 (BR14) (14) high,Oncostatin M/OSM (BR30) (30) high,TREM-1 (BR65) (65) high
S00052,61537.742,28062.930,41073.750,106747.020,165.23999,59.025,122.744995,0.000,0.000,105.905,...,1.08,0.000,0.000,5.285,0.000000,2.230,0.000,1.190,0.0,238.04001
S00109,46735.094,65307.300,29299.984,127422.664,1482.29990,225.220,325.770000,107.785,66.235,0.000,...,0.54,0.000,0.000,43.390,92.945000,16.725,370.050,0.000,0.0,187.57500
S00099,171822.500,181611.300,245606.340,166649.310,1009.95000,0.000,278.320000,0.000,70.500,0.000,...,0.00,0.000,5.960,8.350,0.000000,8.980,0.000,0.000,0.0,955.97000
S00027,151640.360,166586.400,224901.500,153653.530,1570.31510,122.225,342.200000,0.000,69.860,0.000,...,0.00,23.845,2.575,27.840,0.000000,24.465,185.555,1.405,0.0,728.73500
S00124,54641.273,62347.008,51035.367,132546.660,490.92500,0.000,0.000000,0.000,20.110,24.005,...,2.67,0.000,0.000,7.030,9.514999,7.220,0.000,0.000,0.0,171.20999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H00053,48926.680,113737.500,39401.190,45529.870,132.45000,0.000,69.380000,0.000,0.000,310.700,...,0.00,0.000,0.000,1.980,4.310000,0.000,0.000,0.000,0.0,97.23000
H00058,50347.900,55333.880,40959.660,24075.240,123.03000,0.000,74.180000,219.620,0.000,554.430,...,0.00,20.710,0.000,4.050,0.000000,0.000,0.000,0.000,0.0,67.75000
H00085,56736.230,28531.790,58285.810,25120.360,119.25000,0.000,0.000000,328.270,142.210,810.330,...,5.50,0.000,4.290,3.800,0.000000,0.000,0.000,0.000,0.0,126.53000
H00072,61698.570,62108.170,71753.200,36057.590,198.79000,0.000,48.770000,640.530,0.000,660.750,...,0.00,11.150,0.000,6.410,0.000000,0.730,0.000,0.000,0.0,193.38000


In [100]:
luminex_pseudobulks.isna().any().any()

False

In [101]:
luminex_v2 = pd.read_csv("../data_extra_modalities/CBD-KEY-LUMINEX/luminex.csv", sep=',')

In [102]:
luminex_v2

,Row,severity,sex,age,BMI,dexamethasone,CCL18/PARC (BR33) (33) low,Lactoferrin (BR36) (36) high,Lipocalin-2/NGAL (BR21) (21) high,Myeloperoxidase/MPO (BR53) (53) high,...,IFN-alpha (BR63) (63) high,IL-2 (BR43) (43) high,IL-5 (BR53) (53) high,IL-8/CXCL8 (BR48) (48) high,IL-12 p70 (BR56) (56) high,IL-15 (BR52) (52) high,IL-23 (BR76) (76) high,IL-33 (BR14) (14) high,Oncostatin M/OSM (BR30) (30) high,TREM-1 (BR65) (65) high
0,S00029-Ja005E-PMCdb,COVID-critical,F,46,21.4,0,42080.95,15359.95,528930.72,101595.14,...,2.74,0.00,5.71,11.53,15.05,10.50,151.94,1.44,0.00,327.96
1,S00029-Ja001E-PMCdb,COVID-critical,F,46,21.4,0,47991.24,96194.78,467186.02,39638.10,...,0.00,9.42,0.00,14.35,0.00,19.25,0.00,0.00,0.00,728.84
2,S00052-Ja005E-PMCdb,COVID-critical,F,41,30.0,0,76407.98,45607.18,51840.97,88627.44,...,0.00,0.00,0.00,3.74,0.00,0.38,0.00,2.38,0.00,269.00
3,S00109-Ja005E-PMCdb,COVID-critical,F,52,NaN,0,68297.35,64517.42,27631.27,77299.95,...,1.08,0.00,0.00,63.57,96.28,19.37,524.06,0.00,0.00,246.34
4,S00099-Ja005E-PMCdb,COVID-critical,F,52,35.0,0,171822.50,181611.29,245606.35,166649.32,...,0.00,0.00,5.96,8.35,0.00,8.98,0.00,0.00,0.00,955.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,HV0003 Serum,HC (serum),M,61,NaN,0,24539.29,1518500.00,133152.26,133860.98,...,0.00,0.00,0.00,4.55,0.00,0.00,0.00,0.00,0.00,165.83
314,HV0004 Serum,HC (serum),M,31,22.0,0,22783.88,4991600.00,196246.52,184012.97,...,0.00,0.00,0.00,15.32,53.04,9.64,833.92,0.00,2030.36,109.67
315,HV0007 Serum,HC (serum),M,54,NaN,0,30192.52,205670.55,56476.16,67285.56,...,0.00,0.00,0.00,8.79,0.00,0.00,0.00,0.00,0.00,149.44
316,HV0008 Serum,HC (serum),M,53,NaN,0,15566.72,50029.13,44282.33,25543.74,...,0.00,0.00,0.00,4.19,0.00,0.00,0.00,0.00,0.00,74.92
